# Microsimulation of public policies -- Exam : Tax cut subject

#### Professors: Brice Fabre and Sylvain Duchesne

You work as an economist at the Hipeepee Institute, in the country of Peehesseeland. Your job consists in producing ex ante evaluations of fiscal reforms in this country. We are in elections period, and several proposals are made by different parties regarding the tax and benefit system, and you have to provide to the public debate elements related to these proposals.

In [ ]:
#!pip install OpenFisca-PPDLand == 0.3.3 ## YOU NEED TO UNCOMMENT AND RUN THIS LINE ONLY ONCE

In [ ]:
import matplotlib.pyplot as plt  # For graphics
%matplotlib inline

import inspect
import numpy as np  # linear algebra and math
import pandas as pd  # data frames_

from ppdland import CountryTaxBenefitSystem as PSELandTaxBenefitSystem
from ppdland.scenarios import PPDLandSurveyScenario as PSELandSurveyScenario
from ppdland.scenarios import init_single_entity
from ppdland.data import create_input_dataframe

from openfisca_core.model_api import max_
from openfisca_core.rates import marginal_rate

from scipy.optimize import fsolve

from openfisca_core.model_api import *
from ppdland.entities import Individu



%config Completer.use_jedi = False

## 1. Explore the current system

You anticipate that you will have to make soon evaluations in small windows of time. In order to be operational at the right moment, you decide to refresh your memory on what the current system looks like.

We create an input dataframe that we will use later on

For the whole exercise the period concerned is `period = 2020`

### 1.1 What are the main components of the existing tax and benefit system ?

In [ ]:
tax_benefit_system = PSELandTaxBenefitSystem()
TODO
print(variables)

In [ ]:
for variable_name in variables:
    TODO
    try:
        print(variable.name, ":", variable.label)
        TODO
    except Exception:
        # No formula
        pass


### 1.2 Display the features of the existing income tax (tax base, tax schedule).

In [ ]:
from ppdland.variables import variables
TODO

In [ ]:
print(TODO.tax_scale)

### 1.3 Represent budget constraints: draw the net disposable income by pre-tax income for a wage earner. Display also in each graph the first bisector.

In [ ]:
scenario = PSELandTaxBenefitSystem().new_scenario()
scenario = init_single_entity(
    scenario,
    parent1 = {},
    period = '2020',
    axes = [[
        {
            'count': 100,
            'min': 0,
            'max': 9000,
            'name': 'salary',
            }
        ]],
    )
simulation = TODO
TODO
plt.plot(TODO, label = 'disposable_income')
plt.plot(TODO, label = 'first bisector')
plt.xlabel('salary')
plt.legend()

### 1.4 Compute the labour force participation rate (assumption : no unemployment; i.e. every individual finds a job related to her labour supply).

In [ ]:
input_data_frame = create_input_dataframe()
scenario = PSELandSurveyScenario(
    data = dict(input_data_frame = input_data_frame),
    tax_benefit_system = PSELandTaxBenefitSystem(),
    year = 2020,
    )

In [ ]:
participation_rate = TODO
print('Participation rate = {} %'.format(100 * participation_rate))

### 1.5 Compute and represent graphically the mean disposable income by decile of pre-tax income.

In [ ]:
df = scenario.compute_pivot_table(
    TODO,
    period = 2020,
    )
df.stack().reset_index().plot(x = 'pre_tax_income_decile', kind = 'bar')

## 2. Reducing income tax through a reform


The elections get close, and there are some protests of low-income people. The government wants to reduce taxes for these people by reforming the income tax. The tax schedule is unchanged. However, every individual whose income tax is lower than 25 euros will benefit from a tax reduction equals to 18.75-0.75*(income tax before reduction).

### 2.1 Represent the change in income tax implied by this policy: produce a graph showing two curves: the income tax before the reform by pre-tax income level for the case of a single individual wage earner, and the income tax after the reform.

* Write the function that build the reform by completing the cell template



In [ ]:
def build_reform_with_tax_reduction():    
    tax_benefit_system = PSELandTaxBenefitSystem()
    
    class income_tax(Variable):
        def formula(individu, period, parameters):
            salary = individu('salary', period)
            pension = individu('pension', period)
            taxable_income = salary + pension
            tax_scale = parameters(period).tax_scale
            income_tax_before_reduction = tax_scale.calc(taxable_income)
            tax_reduction = TODO
            income_tax_after_reduction = TODO  # you can use max_ to get element-wise maximum of array elements.
            return income_tax_after_reduction
    
    class reformed_tbs(Reform):
        name = u"Adding a tax deduction"

        def apply(self):
            self.update_variable(income_tax)

    return reformed_tbs(tax_benefit_system)

In [ ]:
baseline_scenario = PSELandTaxBenefitSystem().new_scenario()
baseline_scenario = init_single_entity(
    baseline_scenario,
    parent1 = {},
    period = '2020',
    axes = [[
        {
            'count': 10000,
            'min': 0,
            'max': 2000,
            'name': 'salary',
            }
        ]],
    )

reformed_tax_benefit_system = build_reform_with_tax_reduction()
reformed_scenario = reformed_tax_benefit_system.new_scenario()
reformed_scenario = init_single_entity(
    reformed_scenario,
    parent1 = {},
    period = '2020',
    axes = [[
        {
            'count': 10000,
            'min': 0,
            'max': 2000,
            'name': 'salary',
            }
        ]],
    )

TODO

### 2.2 Compute the cost of this reform (in million euros)

Define a more general `cost` function

In [ ]:
def cost():
    reformed_tax_benefit_system = add_bracket_reform()
    scenario = PSELandSurveyScenario(
        data = dict(input_data_frame = input_data_frame),
        TODO,
        year = 2020,
        )

    amount = (
        TODO
        )
    return TODO / 1e6

In [ ]:
cost()

### 2.3 Compute the redistributive effects of the reform: draw a graph showing the mean variation in disposable income by decile of pre-reform pre-tax income

In [ ]:
reformed_tax_benefit_system = build_reform_with_tax_reduction()
scenario = PSELandSurveyScenario(
    data = dict(input_data_frame = input_data_frame),
    TODO,
    year = 2020,
    )
# disposable_income difference after - before by decile
df = scenario.compute_pivot_table(
    TODO
    period = 2020,
    )
df.transpose().reset_index().plot(x = 'pre_tax_income_decile', kind = 'bar')

### 2.4 Interpret the results

Write your answer here

### 2.5 Is there a drawback to this reform ? Describe it if yes

Write your answer here

### 2.6 Your boss asks you whether this figure is the only way to represent the redistributive effects of this reform. What would you answer her/him about other potential representations, the underlying trade-off between these different possibilities, and information you would need to draw such alternative representations?

Write your answer here

## 3. Making the reform budget neutral

Following the publication of your work, some citizens complain that there will be a deficit, and that we do not know in the end who will pay for it. Under pressure, the government decides to increase the top marginal tax rate.

### 3.1 Compute the top-marginal tax rate which makes the reform budget neutral.  

In [ ]:
def build_reform(top_marginal_rate):
    tax_benefit_system = build_reform_with_tax_reduction()
    def modify_top_rate(parameters):
        TODO
        return parameters

    class reformed_tbs(Reform):
        name = u"Setting the top marginal rate to {}".format(top_marginal_rate)
        def apply(self):
            self.modify_parameters(modifier_function = modify_top_rate)

    return reformed_tbs(tax_benefit_system)

In [ ]:
def cost(top_marginal_rate):
    top_marginal_rate = float(top_marginal_rate)
    reformed_tax_benefit_system = TODO
    scenario = PSELandSurveyScenario(
        data = dict(input_data_frame = input_data_frame),
        TODO,
        year = 2020,
        )

    amount = (
        TODO
        )
    return TODO / 1e6

In [ ]:
cost(.5)
budget_balancing_top_marginal_rate = TODO  # you can use fsolve
print(f"The budget-balancing top marginal tax rate is {budget_balancing_top_marginal_rate}")

### 3.2 Represent the change in income tax implied by this policy: produce a graph showing two curves: the income tax before the reform by pre-tax income level for the case of a single individual wage earner, and the income tax after the reform (reform = the new tax reduction + the new top tax rate).

In [ ]:
baseline_scenario = PSELandTaxBenefitSystem().new_scenario()
baseline_scenario = init_single_entity(
    baseline_scenario,
    parent1 = {},
    period = '2020',
    axes = [[
        {
            'count': 10000,
            'min': 0,
            'max': 2000,
            'name': 'salary',
            }
        ]],
    )

TODO

### 3.3 Compute the redistributive effects of the reform: draw a graph showing the mean variation in disposable income by decile of pre-reform pre-tax income.

In [ ]:
TODO

## 4. Labour supply behavioural responses along the extensive margin

Now, your boss says you that behavioural responses are, according to the recent papers, mainly along the extensive margin. Your boss asks you to evaluate the impact of this reform by taking into account only responses at the extensive margin, taking an elasticity of 0.6. Reminder: this elasticity is equal to the variation of the probability to work when the difference of disposable income between working and not working increases by 1%. For simplicity, we assume that the initial probability to work is the same for all individuals and equal to the share of workers in the labour force.

### 4.1 What do you think about this strategy (the extensive margin is the most relevant) regarding the existing literature?

Write your answer here

### 4.2 Explain in plain English in which direction behavioural responses in this setting will go.

Write your answer here

### 4.3 Implement the reform that takes into account behavioural responses. Produce summary statistics on the variation in participation: the share of former non-participants who participate now (if some) and the share of former participant who still participate (if some)

In [ ]:
def participation_extension(top_marginal_rate):
    tax_benefit_system = build_reform(top_marginal_rate)

    class disposable_income_participation_variation(Variable):
        definition_period = YEAR
        entity = Individu
        value_type = float

        def formula(individu, period, parameters):
            # If an individual starts working, she is supposed to earn her potential salary
            # it is nonzero only for potential and actual wage earners
            # salary is nonzero only for actual wage earners
            potential_salary = individu('potential_salary', period)

            def compute_income_tax_after_reduction(salary_):
                TODO
                return TODO

            disposable_income_participation_variation = where(
                potential_salary > 0,
                TODO,
                0
                )
            return disposable_income_participation_variation

    class participation_adjusted_salary(Variable):
        definition_period = YEAR
        entity = Individu
        value_type = float

        def formula_2020(individu, period, parameters):
            np.random.seed(seed = 42)
            salary = individu('salary', period)
            potential_salary = individu('potential_salary', period)

            disposable_income_participation_variation = individu('disposable_income_participation_variation', period)
            pre_reform_disposable_income_participation_variation = where(
                potential_salary > 0,
                TODO,
                0
                )
            eta = .6
            delta_DIPV_over_DIPV = where(
                potential_salary > 0,
                TODO,
                0,
                )
            initial_participation_probability = sum(salary > 0) / sum(potential_salary > 0)
            delta_participation_probability = eta * TODO * TODO
            new_participation_probability = initial_participation_probability + delta_participation_probability
            probability_still_in_work_among_initial_workers = TODO
            still_in_work = (
                (salary > 0)
                * (np.random.uniform(size = len(salary)) < probability_still_in_work_among_initial_workers)
                )
            print("Share of initial workers still working: ", sum(still_in_work) / sum(salary > 0))

            probability_newly_employed = TODO
            newly_employed = (
                (salary == 0) * (potential_salary > 0)
                * (np.random.uniform(size = len((salary == 0) * (potential_salary > 0))) < probability_newly_employed)
                )
            print("Share of initial non-participants entering into participation: ", sum(newly_employed) / sum((salary == 0) * (potential_salary > 0)))


            return where(
                still_in_work + newly_employed,  # + is or for boolean
                potential_salary,
                0
                )

    class income_tax(Variable):
        definition_period = YEAR
        entity = Individu
        value_type = float

        def formula(individu, period, parameters):
            TODO

    class disposable_income(Variable):
        definition_period = YEAR
        entity = Individu
        value_type = float

        def formula_2020(individu, period):
            TODO

    class reformed_tbs(Reform):
        name = "Reformed tax benefit system"

        def apply(self):
            self.add_variable(TODO)
            self.add_variable(TODO)
            self.update_variable(TODO)
            self.update_variable(TODO)

    return reformed_tbs(tax_benefit_system)

In [ ]:
participation_adjusted_reformed_tax_benefit_system = TODO
scenario = PSELandSurveyScenario(
    data = dict(input_data_frame = input_data_frame),
    TODO,
    TODO,
    year = 2020,
    )

In [ ]:
salary = scenario.calculate_variable("salary", period = 2020)
potential_salary = scenario.calculate_variable("potential_salary", period = 2020)
participation_adjusted_salary = scenario.calculate_variable("participation_adjusted_salary", period = 2020)
print("Share of initial non-participants entering into participation: {} %".format(
    100 * TODO
    ))
print("Share of initial participants who still participate:  {} %".format(
    100 * TODO
    ))


### 4.4 Compute the cost of the reform after taking into account these behavioural responses.

In [ ]:
TODO

### 4.5 Compute the new labour force participation rate.

In [ ]:
previous_participation_rate = (
    TODO
    )
print('Initial participation rate = {} %'.format(100 * participation_rate))

In [ ]:
participation_rate = (
    TODO
    )
print('Participation rate = {} %'.format(100 * participation_rate))

### 4.6 Compute the redistributive effects of the reform: draw a graph showing the mean variation in disposable income by decile of pre-reform pre-tax income.

In [ ]:
TODO

### 4.7 Comment all these results.

Write your answer here

# 5. About the future

### Your boss says that now, you have to publish all your work, because journalists are getting crazy for not having the results of your evaluation. However, he suggests that you need to have together a meeting to talk about other ways to microsimulate the impacts of reforms of these kinds, in order to anticipate for future work.


Write your answer here